## Importing python libraries and starting a sas session locally on windows

In [9]:
import os
import json
from pprint import pprint
from airtable import Airtable
import pandas as pd
import saspy
sas = saspy.SASsession(cfgname='winlocal')

SAS Connection established. Subprocess id is 2756



## Set api key as a global environment variable in Windows settings
![Finding global variables](environment_variable.png)
![Setting api key for Airtable](api_key.png)

**- which database - base key**   
**- which table - yield**  
**- api key - stored locally as an environement variable in windows**

In [10]:
base_key = 'appInRX5iM5iijXuY'
table_name = 'yield'
wheat_yield = Airtable(base_key, table_name, api_key=os.environ['AIRTABLE_API_KEY'])
print(wheat_yield)

<Airtable table:yield>


## Call airtable library and get every record in the previously identified table

Stuff in parentheses after dataframe call is a mystery.  Found at https://github.com/gtalarico/airtable-python-wrapper/blob/master/Airtable.ipynb
Delete the Date/Time column because sas hates it

In [12]:
examplerecords = Airtable.get_all()    
wheat_yield_df = pd.DataFrame.from_records((r['fields'] for r in examplerecords))
wheat_yield_df = wheat_yield_df.drop(columns='Date/Time')

NameError: name 'self' is not defined

In [ ]:
wheat_yield_df.head(3)

## Declare a sas libary on your local machine.  This is really just like a temp file to move data from pandas to sas.
Windows normal string vs raw string, see: https://stackoverflow.com/questions/37400974/unicode-error-unicodeescape-codec-cant-decode-bytes-in-position-2-3-trunca

In [ ]:
sas.saslib('example', path=r"C:\Users\screberg\Documents\My SAS Files") 

# Move data from pandas to SAS

In [ ]:
sas_yield = sas.df2sd(df=wheat_yield_df,table='sas_yield',libref='example')
sas_yield.head(3)

## This lets us look at some basic statistics for the variable of interest  
dealing with temporary problem - all the data came in as character

In [ ]:
%%SAS
libname example 'C:\Users\screberg\Documents\My SAS Files';
data example.sas_yield; set example.sas_yield;
   Tstwght_num = input(Tstwght, 8.); run;
proc univariate data=example.sas_yield; 
Var Tstwght_num; 
Histogram;
run;